In [1]:
from pixell import enmap, enplot, utils, curvedsky
from cosmikyu import maps
from soapack import interfaces as soint
import healpy as hp
import numpy as np
import os

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
output_dir = '/simons/scratch/dhan1/cosmikyu/data'
def output_path(x):
    return os.path.join(output_dir, x)

In [3]:
sehgal_sim_path = soint.dconfig['actsims']['sehgal09_path']
SH = maps.SehgalSky2010(sehgal_sim_path, data_type='alm')

res = 0.5*utils.arcmin
alm = SH.get_cib(148)
fshape, fwcs = enmap.fullsky_geometry(res)
template = maps.get_template(fshape, fwcs)
fmap = curvedsky.alm2map(alm, template)

del template


In [4]:
meta = np.array([np.mean(fmap), np.std(fmap), np.max(fmap), np.min(fmap)])
np.savetxt(output_path("ir_pts/148_meta.txt"), meta, header="mean,std,max,min", comments="#[IR Pts f148 0.5 arcmin res]   ")

In [5]:
stamp_width = stamp_height = 2.5*utils.degree
nx = int(np.ceil(stamp_width/res))
ny = int(np.ceil(stamp_height/res))
shape = (ny, nx)
print("Nx: {}, Ny: {}".format(nx, ny))

Nx: 300, Ny: 300


In [6]:
shape, wcs, _ = maps.get_geometry(np.pi, np.pi, stamp_width, stamp_height, shape=shape)
template = maps.get_template(shape, wcs)

In [11]:
for sim_idx in np.arange(10000):
    if sim_idx %100 == 0: print(sim_idx)
    try:
        shape, wcs, _ = maps.rand_geometry(stamp_width, stamp_height, shape=shape, seed=sim_idx)
        #stamp = enmap.extract(fmap, shape=shape, wcs=wcs)
    except:
        print(wcs)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [8]:
wcs, _, _ = maps.get_geometry(1.3520746650524709,1.7241484721289566 , stamp_width, stamp_width, res=0.5)

enmap.wcsutils.is_compatible(fwcs, fwcs)

True

In [9]:
mean, std, max_val, min_val 

NameError: name 'mean' is not defined

In [ ]:
maps.rand_geometry??